### 필요한 라이브러리 불러오기

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

from matplotlib import pyplot as plt
import seaborn as sn
import pandas as pd
import numpy as np
import os

device = 'cuda'

### 데이터셋(Dataset) 다운로드 및 불러오기

In [2]:
import torchvision
import torchvision.transforms as transforms

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

# 일반적으로 많이 사용되는 입력 전처리(preprocessing) 테크닉
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4), # 패딩 이후 랜덤하게 잘라서 뽑기
    transforms.RandomHorizontalFlip(), # 좌우 반전 (MNIST 말고 CIFAR10에서는 효과적)
    transforms.ToTensor(),
    normalize, # 실험 결과, 입력 정규화(input normalization)가 성능에 크게 영향을 미치지는 않음
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    normalize,
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

100%|██████████| 170498071/170498071 [00:07<00:00, 24002139.93it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


여기서 mean과 std는 각각 평균과 표준편차를 나타내며, 각 값은 RGB 채널에 대응됩니다.


R 채널의 평균: 0.485, 표준편차: 0.229


G 채널의 평균: 0.456, 표준편차: 0.224


B 채널의 평균: 0.406, 표준편차: 0.225


이 값들은 ImageNet 데이터셋에서 학습된 모델들을 위한 일반적인 평균 및 표준편차 값입니다. CIFAR-10과 같은 다른 데이터셋에 대해서도 이 값을 사용하는 것이 일반적입니다, 왜냐하면 많은 사전 학습된 모델들이 이러한 값으로 정규화된 데이터에 대해 학습되었기 때문입니다.


정규화를 수행하는 이유는 다음과 같습니다:
- 수렴 속도 개선: 정규화된 입력은 학습 알고리즘이 더 빠르게 수렴하도록 도와줍니다.
- 가중치 초기화: 정규화된 입력은 가중치 초기화에 덜 민감하게 만들어줍니다.
- 옵티마이저 최적화: 정규화는 옵티마이저가 더 효율적으로 최적화를 수행하도록 도와줍니다.


이러한 이유로, 정규화는 딥러닝에서 이미지 데이터를 처리할 때 거의 항상 사용되는 전처리 단계입니다. - GPT

In [3]:
def train(net, epoch, optimizer, criterion, train_loader):
  print('[Train epoch : %d]' % epoch )
  net.train()
  train_loss = 0
  correct = 0
  total = 0
  for batch_idx, (inputs, targets) in enumerate(train_loader):
    inputs, targets = inputs.to(device), targets.to(device)
    optimizer.zero_grad()

    benign_outputs = net(inputs)
    loss = criterion(benign_outputs, targets)
    loss.backward()

    optimizer.step()
    train_loss += loss.item()
    _, predicted = benign_outputs.max(1)

    total += targets.size(0)
    correct += predicted.eq(targets).sum().item()

  print('Train accuarcy:', 100. * correct / total)
  print('Train average loss:', train_loss / total)
  return (100. * correct / total, train_loss / total)

def evaluate(net, epoch, file_name, data_loader, info):
    print('[ Evaluate epoch: %d ]' % epoch)
    print("Dataset:", info)

    net.eval()
    test_loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(data_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        test_loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('Accuarcy:', 100. * correct / total)
    print('Average loss:', test_loss / total)
    return (100. * correct / total, test_loss / total)

### Confusion Matrix 정의

In [4]:
def get_confusion_matrix(net, num_classes, data_loader):
  confusion_matrix = torch.zeros(num_classes, num_classes)

  net.eval()

  for batch_idx, (inputs, targets) in enumerate(data_loader):
    inputs, targets = inputs.to(device), targets.to(device)

    outputs = net(inputs)
    _, predicted = outputs.max(1)

    for t, p in zip(targets.view(-1), predicted.view(-1)):
      confusion_matrix[t.long(), p.long()] += 1

  return confusion_matrix

### ResNet 모델 정의

- ResNet 논문에서 제안된 CIFAR-10 전용 아키텍처를 따릅니다.
- CIFAR-10 데이터셋을 위한 아키텍처의 파라미터가 훨씬 적습니다.
- ImageNet 데이터셋을 위한 아키텍처와는 차이가 있습니다.

In [ ]:
class LambdaLayer(nn.Module):
  def __init__(self, lambd):
    super(LambdaLayer, self).__init__()
    self.lambd = lambd

  def forward(self, x):
    return self.lambd(x)

#ResNet을 위한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
  def __init__(self, in_planes, planes, stride=1):
    super(BasicBlock, self).__init__()

    #3x3 필터를 사용(너비와 높이를 줄일때는 stride 값 조절)
    self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
